# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_id_text =[]
    source_sentences = source_text.split('\n')
    for source_raw_sentence in source_sentences:
        final_sentence = source_raw_sentence.split()
        lettuce = [source_vocab_to_int[word]for word in final_sentence]
        source_id_text.append(lettuce)
    print(source_id_text)
    
    eos =[target_vocab_to_int['<EOS>']]
    target_id_text =[]
    target_sentences = target_text.split('\n')
    for target_raw_sentence in target_sentences:
        final_sentence = target_raw_sentence.split()
        lettuce = [target_vocab_to_int[word]for word in final_sentence]+eos
        target_id_text.append(lettuce)
    print(target_id_text)
    
    return source_id_text,target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

[[16, 13, 14, 32, 5, 24, 22, 23, 17, 15, 14, 6, 25, 9, 19], [20, 10, 26, 14, 4, 12, 24, 27, 23, 17, 15, 14, 4, 31, 25, 7, 19], [18, 14, 4, 5, 24, 8, 23, 17, 15, 14, 4, 28, 25, 11, 19], [20, 10, 26, 14, 32, 29, 24, 11, 23, 17, 15, 14, 21, 25, 30, 19]]
[[13, 11, 6, 25, 9, 21, 18, 22, 29, 16, 12, 6, 10, 31, 19, 15, 1], [24, 27, 6, 7, 30, 31, 17, 29, 16, 12, 5, 26, 31, 23, 15, 1], [14, 6, 7, 9, 31, 4, 29, 16, 12, 6, 7, 32, 31, 28, 15, 1], [24, 27, 6, 25, 20, 31, 28, 29, 16, 12, 33, 30, 31, 8, 15, 1]]
Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    
    input = tf.placeholder(tf.int32,[None, None],name = "input")
    targets = tf.placeholder(tf.int32,[None, None],name = "targets")
    learning_rate = tf.placeholder(tf.float32,name = "learning_rate")
    keep_prob = tf.placeholder(tf.float32,name = "keep_prob")
    target_sequence_length = tf.placeholder(tf.int32, (None,), name = "target_sequence_length")
    max_target_len = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32,(None,),name = "source_sequence_length")
    
    return input, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    preprocessed_target_data = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return preprocessed_target_data
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop

    enc_cell_with_dropout=tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell_with_dropout, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    return enc_output, enc_state
    
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input,target_sequence_length, name="training_helper")
    seq2seq_basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,training_helper,encoder_state,output_layer)
    BasicDecoderOutput,_,_ = tf.contrib.seq2seq.dynamic_decode(seq2seq_basic_decoder,impute_finished=True,maximum_iterations=max_summary_length)
    return BasicDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    
    emb_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,start_tokens,end_of_sequence_id)
    seq2seq_basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,emb_helper,encoder_state,output_layer)
    result,_,_ = tf.contrib.seq2seq.dynamic_decode(seq2seq_basic_decoder,impute_finished=True,maximum_iterations=max_target_sequence_length)
    return result



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
 
    output_layer = Dense(target_vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

    with tf.variable_scope("decode") as decoding_scope:
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input,target_sequence_length, max_target_sequence_length, output_layer, keep_prob)

    with tf.variable_scope('decode', reuse=True) as decoding_scope:
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],target_vocab_to_int['<EOS>'], max_target_sequence_length,target_vocab_size, output_layer, batch_size, keep_prob)

    return training_decoder_output, inference_decoder_output
    
    
    
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    enc_input, enc_state = encoding_layer(input_data,rnn_size,num_layers,keep_prob,source_sequence_length,source_vocab_size,enc_embedding_size)
    preprocessed_target_data = process_decoder_input(target_data,target_vocab_to_int,batch_size)
    training_basic_decoder_output,inference_basic_decoder_output = decoding_layer(target_data,enc_state,target_sequence_length, max_target_sentence_length,rnn_size,num_layers,target_vocab_to_int,target_vocab_size,batch_size, keep_prob, dec_embedding_size)
    return training_basic_decoder_output,inference_basic_decoder_output 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 1
# Batch Size
batch_size = 10
# RNN Size
rnn_size = 2
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.02
# Dropout Keep Probability
keep_probability = 0.3
display_step = 5

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/13786 - Train Accuracy: 0.1778, Validation Accuracy: 0.0529, Loss: 5.6746
Epoch   0 Batch   10/13786 - Train Accuracy: 0.1667, Validation Accuracy: 0.0529, Loss: 5.3477
Epoch   0 Batch   15/13786 - Train Accuracy: 0.2056, Validation Accuracy: 0.0529, Loss: 5.0263
Epoch   0 Batch   20/13786 - Train Accuracy: 0.3150, Validation Accuracy: 0.0529, Loss: 4.6669
Epoch   0 Batch   25/13786 - Train Accuracy: 0.1789, Validation Accuracy: 0.0529, Loss: 4.3928
Epoch   0 Batch   30/13786 - Train Accuracy: 0.1263, Validation Accuracy: 0.0529, Loss: 4.3439
Epoch   0 Batch   35/13786 - Train Accuracy: 0.2158, Validation Accuracy: 0.0529, Loss: 4.1370
Epoch   0 Batch   40/13786 - Train Accuracy: 0.2333, Validation Accuracy: 0.0529, Loss: 4.0757
Epoch   0 Batch   45/13786 - Train Accuracy: 0.1000, Validation Accuracy: 0.0529, Loss: 4.0456
Epoch   0 Batch   50/13786 - Train Accuracy: 0.1056, Validation Accuracy: 0.0529, Loss: 4.0841
Epoch   0 Batch   55/13786 - Train Accuracy: 0.272

Epoch   0 Batch  440/13786 - Train Accuracy: 0.1500, Validation Accuracy: 0.1059, Loss: 2.9154
Epoch   0 Batch  445/13786 - Train Accuracy: 0.1500, Validation Accuracy: 0.1059, Loss: 2.8141
Epoch   0 Batch  450/13786 - Train Accuracy: 0.1300, Validation Accuracy: 0.0824, Loss: 2.8370
Epoch   0 Batch  455/13786 - Train Accuracy: 0.0833, Validation Accuracy: 0.1059, Loss: 2.9929
Epoch   0 Batch  460/13786 - Train Accuracy: 0.1444, Validation Accuracy: 0.1059, Loss: 2.8849
Epoch   0 Batch  465/13786 - Train Accuracy: 0.1444, Validation Accuracy: 0.1059, Loss: 2.9973
Epoch   0 Batch  470/13786 - Train Accuracy: 0.1176, Validation Accuracy: 0.1059, Loss: 2.9347
Epoch   0 Batch  475/13786 - Train Accuracy: 0.1900, Validation Accuracy: 0.1059, Loss: 2.5651
Epoch   0 Batch  480/13786 - Train Accuracy: 0.1778, Validation Accuracy: 0.0824, Loss: 2.9318
Epoch   0 Batch  485/13786 - Train Accuracy: 0.1722, Validation Accuracy: 0.0824, Loss: 2.9319
Epoch   0 Batch  490/13786 - Train Accuracy: 0.155

Epoch   0 Batch  875/13786 - Train Accuracy: 0.0222, Validation Accuracy: 0.0000, Loss: 2.6285
Epoch   0 Batch  880/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 2.5489
Epoch   0 Batch  885/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0000, Loss: 2.3961
Epoch   0 Batch  890/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0000, Loss: 2.4737
Epoch   0 Batch  895/13786 - Train Accuracy: 0.0176, Validation Accuracy: 0.0176, Loss: 2.8472
Epoch   0 Batch  900/13786 - Train Accuracy: 0.0059, Validation Accuracy: 0.0176, Loss: 2.6064
Epoch   0 Batch  905/13786 - Train Accuracy: 0.0100, Validation Accuracy: 0.0176, Loss: 2.4051
Epoch   0 Batch  910/13786 - Train Accuracy: 0.0167, Validation Accuracy: 0.0176, Loss: 2.6773
Epoch   0 Batch  915/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0176, Loss: 2.3605
Epoch   0 Batch  920/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0176, Loss: 2.5534
Epoch   0 Batch  925/13786 - Train Accuracy: 0.016

Epoch   0 Batch 1310/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0176, Loss: 2.3680
Epoch   0 Batch 1315/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0176, Loss: 2.1668
Epoch   0 Batch 1320/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0176, Loss: 2.3819
Epoch   0 Batch 1325/13786 - Train Accuracy: 0.0211, Validation Accuracy: 0.0176, Loss: 2.2143
Epoch   0 Batch 1330/13786 - Train Accuracy: 0.0222, Validation Accuracy: 0.0000, Loss: 2.2853
Epoch   0 Batch 1335/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0000, Loss: 2.3084
Epoch   0 Batch 1340/13786 - Train Accuracy: 0.0111, Validation Accuracy: 0.0000, Loss: 2.1497
Epoch   0 Batch 1345/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 2.0930
Epoch   0 Batch 1350/13786 - Train Accuracy: 0.0222, Validation Accuracy: 0.0000, Loss: 2.1776
Epoch   0 Batch 1355/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 2.1555
Epoch   0 Batch 1360/13786 - Train Accuracy: 0.000

Epoch   0 Batch 1745/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0176, Loss: 2.1478
Epoch   0 Batch 1750/13786 - Train Accuracy: 0.0158, Validation Accuracy: 0.0176, Loss: 1.9933
Epoch   0 Batch 1755/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0176, Loss: 2.1648
Epoch   0 Batch 1760/13786 - Train Accuracy: 0.0111, Validation Accuracy: 0.0176, Loss: 2.1131
Epoch   0 Batch 1765/13786 - Train Accuracy: 0.0111, Validation Accuracy: 0.0176, Loss: 2.0270
Epoch   0 Batch 1770/13786 - Train Accuracy: 0.0200, Validation Accuracy: 0.0176, Loss: 1.8921
Epoch   0 Batch 1775/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0176, Loss: 2.0363
Epoch   0 Batch 1780/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0176, Loss: 2.0563
Epoch   0 Batch 1785/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0176, Loss: 2.1691
Epoch   0 Batch 1790/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0176, Loss: 2.1716
Epoch   0 Batch 1795/13786 - Train Accuracy: 0.015

...

Epoch   0 Batch 11730/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0059, Loss: 1.0596
Epoch   0 Batch 11735/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0059, Loss: 0.8783
Epoch   0 Batch 11740/13786 - Train Accuracy: 0.0100, Validation Accuracy: 0.0000, Loss: 0.9040
Epoch   0 Batch 11745/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0000, Loss: 1.0585
Epoch   0 Batch 11750/13786 - Train Accuracy: 0.0150, Validation Accuracy: 0.0059, Loss: 1.0537
Epoch   0 Batch 11755/13786 - Train Accuracy: 0.0211, Validation Accuracy: 0.0059, Loss: 0.8509
Epoch   0 Batch 11760/13786 - Train Accuracy: 0.0111, Validation Accuracy: 0.0000, Loss: 0.8980
Epoch   0 Batch 11765/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 0.9644
Epoch   0 Batch 11770/13786 - Train Accuracy: 0.0111, Validation Accuracy: 0.0059, Loss: 0.9812
Epoch   0 Batch 11775/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 0.8657
Epoch   0 Batch 11780/13786 - Train Accu

Epoch   0 Batch 12160/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0000, Loss: 0.8792
Epoch   0 Batch 12165/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 0.9943
Epoch   0 Batch 12170/13786 - Train Accuracy: 0.0150, Validation Accuracy: 0.0000, Loss: 0.8654
Epoch   0 Batch 12175/13786 - Train Accuracy: 0.0158, Validation Accuracy: 0.0000, Loss: 0.9683
Epoch   0 Batch 12180/13786 - Train Accuracy: 0.0050, Validation Accuracy: 0.0000, Loss: 0.8743
Epoch   0 Batch 12185/13786 - Train Accuracy: 0.0050, Validation Accuracy: 0.0000, Loss: 0.8559
Epoch   0 Batch 12190/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 0.8814
Epoch   0 Batch 12195/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0000, Loss: 1.1481
Epoch   0 Batch 12200/13786 - Train Accuracy: 0.0222, Validation Accuracy: 0.0000, Loss: 0.8457
Epoch   0 Batch 12205/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0000, Loss: 0.8077
Epoch   0 Batch 12210/13786 - Train Accu

Epoch   0 Batch 12590/13786 - Train Accuracy: 0.0167, Validation Accuracy: 0.0059, Loss: 0.8725
Epoch   0 Batch 12595/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0059, Loss: 1.0624
Epoch   0 Batch 12600/13786 - Train Accuracy: 0.0235, Validation Accuracy: 0.0059, Loss: 1.0797
Epoch   0 Batch 12605/13786 - Train Accuracy: 0.0211, Validation Accuracy: 0.0000, Loss: 0.9367
Epoch   0 Batch 12610/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0000, Loss: 1.0794
Epoch   0 Batch 12615/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0000, Loss: 1.0236
Epoch   0 Batch 12620/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0000, Loss: 1.0065
Epoch   0 Batch 12625/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0000, Loss: 0.8057
Epoch   0 Batch 12630/13786 - Train Accuracy: 0.0250, Validation Accuracy: 0.0059, Loss: 0.9327
Epoch   0 Batch 12635/13786 - Train Accuracy: 0.0050, Validation Accuracy: 0.0000, Loss: 0.8679
Epoch   0 Batch 12640/13786 - Train Accu

Epoch   0 Batch 13020/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0000, Loss: 0.8762
Epoch   0 Batch 13025/13786 - Train Accuracy: 0.0050, Validation Accuracy: 0.0000, Loss: 0.8068
Epoch   0 Batch 13030/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0000, Loss: 0.9766
Epoch   0 Batch 13035/13786 - Train Accuracy: 0.0167, Validation Accuracy: 0.0059, Loss: 0.8241
Epoch   0 Batch 13040/13786 - Train Accuracy: 0.0111, Validation Accuracy: 0.0059, Loss: 1.0490
Epoch   0 Batch 13045/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0059, Loss: 0.8231
Epoch   0 Batch 13050/13786 - Train Accuracy: 0.0300, Validation Accuracy: 0.0059, Loss: 0.8891
Epoch   0 Batch 13055/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0059, Loss: 0.8713
Epoch   0 Batch 13060/13786 - Train Accuracy: 0.0056, Validation Accuracy: 0.0059, Loss: 0.8648
Epoch   0 Batch 13065/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0059, Loss: 0.8339
Epoch   0 Batch 13070/13786 - Train Accu

Epoch   0 Batch 13450/13786 - Train Accuracy: 0.0211, Validation Accuracy: 0.0059, Loss: 0.9192
Epoch   0 Batch 13455/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0000, Loss: 1.1241
Epoch   0 Batch 13460/13786 - Train Accuracy: 0.0167, Validation Accuracy: 0.0059, Loss: 1.0341
Epoch   0 Batch 13465/13786 - Train Accuracy: 0.0095, Validation Accuracy: 0.0059, Loss: 0.9153
Epoch   0 Batch 13470/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0059, Loss: 0.9945
Epoch   0 Batch 13475/13786 - Train Accuracy: 0.0050, Validation Accuracy: 0.0059, Loss: 0.7858
Epoch   0 Batch 13480/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0059, Loss: 0.8513
Epoch   0 Batch 13485/13786 - Train Accuracy: 0.0000, Validation Accuracy: 0.0000, Loss: 0.9638
Epoch   0 Batch 13490/13786 - Train Accuracy: 0.0053, Validation Accuracy: 0.0059, Loss: 0.9550
Epoch   0 Batch 13495/13786 - Train Accuracy: 0.0105, Validation Accuracy: 0.0000, Loss: 0.9988
Epoch   0 Batch 13500/13786 - Train Accu

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [22]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    list_of_words_id = []
    lowercase_sentence = sentence.lower()
    prepared_sentence = lowercase_sentence.split()
    substitute = [vocab_to_int["<UNK>"]]
    for word in prepared_sentence:
        if word in list(vocab_to_int.keys()):
            list_of_words_id = list_of_words_id  + [vocab_to_int[word]]
        if word not in list(vocab_to_int.keys()):
            list_of_words_id = list_of_words_id  + substitute    
    return list_of_words_id


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [26]:
translate_sentence = 'new jersey is sometimes quiet during autumn , and it is snowy in april .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [37, 200, 97, 171, 122, 116, 215, 219, 41, 201, 97, 15, 221, 83, 102]
  English Words: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', ',', 'and', 'it', 'is', 'snowy', 'in', 'april', '.']

Prediction
  Word Ids:      [126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126]
  French Words: la la la la la la la la la la la la la la la la la la la la la la la la la la la la la la


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.